In [1]:
import codecs, json, glob, os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import tensorflow as tf
from sklearn.metrics import confusion_matrix, jaccard_score, classification_report
import numpy as np
from sklearn.metrics import f1_score
import pandas as pd
import matplotlib.pyplot as plt

from utils.utils import label_mapping
from utils.image_processing import read_resize
from config import TRAIN_DATA_PATH
from utils.plotting import plot_image_label_prediction, plot_uncertainty_heatmaps, \
    plot_uncertainty_statistics, plot_image, plot_label, plot_predictions, plot_idv_records

result_json_dir = "/home/olle/PycharmProjects/LODE/workspace/feature_segmentation/opt_ensemble"

configured_labels = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 13, 14, 15]


## load json files to evaluate

In [ ]:
def read_decode_json(file_path):
    obj_text = codecs.open(file_path, 'r', encoding='utf-8').read()
    
    json_result_file = json.loads(obj_text)

    json_result_file["image"] = np.array(json_result_file["image"])
    json_result_file["label"] = np.array(json_result_file["label"])
    json_result_file["prediction"] = np.array(json_result_file["prediction"])
    return json_result_file
    
json_result_paths = glob.glob(result_json_dir + "/*.json")

result_jsons = {}
for json_file_path in json_result_paths:
    record = json_file_path.split("/")[-1].replace(".json", "")
    result_jsons[record] = read_decode_json(json_file_path)

## plot records for publication

In [ ]:
records = result_jsons.keys()

for record in records:
    record_log = result_jsons[record]
    data = [record_log["image"], record_log["label"], record_log["prediction"]]
    
    # all plotting functions
    plot_image_label_prediction(data,model_dir = result_json_dir, filename=record, mode="test")
    plot_image(record_log["image"], model_dir = result_json_dir, filename=record, mode="test")
    plot_label(record_log["label"], model_dir = result_json_dir, filename=record, mode="test")
    plot_predictions([record_log["prediction"]], model_dir = result_json_dir, filename=record, mode="test")
    
    plt.close()

## create results per record result logs

In [ ]:
records = result_jsons.keys()

f1_record_log = {}
f1_total_log = {}

all_labels = []
all_predictions = []

for record in records:
    record_log = result_jsons[record]
    
    labels = record_log["label"].flatten()
    predictions = record_log["prediction"].flatten()
    
    all_labels.extend(labels.tolist())
    all_predictions.extend(predictions.tolist())
    
    f1_record_log[record] = f1_score(labels, 
                         predictions, 
                         average = None, 
                         labels = configured_labels, 
                         zero_division=1)

# calculate for each log
f1_total_log = f1_score(all_labels, 
                        all_predictions, 
                        average = None, 
                        labels = configured_labels, 
                        zero_division=1)

# convert to data frames for plotting
total_f1_pd = pd.DataFrame(f1_total_log).T.replace(1.0,np.NaN)
record_f1_pd = pd.DataFrame(f1_record_log).T.replace(1.0,np.NaN)

In [ ]:
total_f1_pd

## Inter doctor variation analysis

In [2]:
# set env variables
idv_json_dir = "/home/olle/PycharmProjects/LODE/workspace/feature_segmentation/opt_ensemble/idv"

evaluators = ["ben", "johannes", "michael", "concensus"]

In [3]:
def read_decode_json(file_path):
    obj_text = codecs.open(file_path, 'r', encoding='utf-8').read()
    
    json_result_file = json.loads(obj_text)
    json_result_file["image"] = np.array(json_result_file["image"])
    
    # add all evaluators results
    for evaluator in evaluators:
        json_result_file[f"{evaluator}_mask"] = np.array(json_result_file[f"{evaluator}_mask"])

    json_result_file["prediction"] = np.array(json_result_file["prediction"])
    return json_result_file
    
json_result_paths = glob.glob(idv_json_dir + "/*.json")

result_jsons = {}
for json_file_path in json_result_paths:
    record = json_file_path.split("/")[-1].replace(".json", "")
    result_jsons[record] = read_decode_json(json_file_path)

## plot all records

In [5]:
idv_json_dir

'/home/olle/PycharmProjects/LODE/workspace/feature_segmentation/opt_ensemble/idv'

In [6]:
records = result_jsons.keys()

for record in records:
    record_log = result_jsons[record]
    data = [record_log["image"], record_log["concensus_mask"], record_log["prediction"]]
    
    idv_data = [record_log["image"], record_log["concensus_mask"], 
                record_log["ben_mask"], record_log["johannes_mask"], 
                record_log["michael_mask"], record_log["prediction"]]

    # all plotting functions
    plot_image_label_prediction(data,model_dir = idv_json_dir, filename=record, mode="test")
    plot_image(record_log["image"], model_dir = idv_json_dir, filename=record, mode="test")
    
    plot_idv_records(idv_data, model_dir = idv_json_dir, filename=record, mode="idv_test")
     # add all evaluators results
    for evaluator in evaluators:
        plot_label(record_log[f"{evaluator}_mask"], 
                   model_dir = idv_json_dir, 
                   filename=f"{evaluator}_{record}", 
                   mode="test")

    plot_predictions([record_log["prediction"]], model_dir = idv_json_dir, filename=record, mode="test")

## extract results

In [ ]:
records = result_jsons.keys()

f1_record_log = {}
f1_total_log = {}

labels_predictions = {"concensus": []}

for evaluator in evaluators + ["prediction"]:
    labels_predictions[evaluator] = []
        
for record in records:
    record_log = result_jsons[record]
    
    labels_predictions["prediction"].extend(record_log[f"prediction"].flatten().tolist())

    for evaluator in evaluators:
        labels_predictions[evaluator].extend(record_log[f"{evaluator}_mask"].flatten().tolist())


In [ ]:
idv_logs = {}
for evaluator in evaluators + ["prediction"]:
    # calculate for each log
    f1_total_log = f1_score(labels_predictions["concensus"], 
                            labels_predictions[evaluator], 
                            average = None, 
                            labels = configured_labels, 
                            zero_division=1)

    idv_logs[evaluator] = f1_total_log
    
# convert to data frames for plotting
total_f1_pd = pd.DataFrame(idv_logs).T.replace(1.0,np.NaN)


In [ ]:
pd.DataFrame(idv_logs)

In [ ]:
total_f1_pd